In [1]:
from geopy.distance import geodesic
from geopy.point import Point
from geopy.geocoders import Nominatim
import overpy
from decimal import Decimal
import osmapi
from tqdm import tqdm
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import time
import geopy
import warnings

warnings.filterwarnings("ignore")

In [2]:
shop_flc = pd.read_excel(r'D:\SHOP_LC.xlsx') #lấy dữ liệu shop long châu
get_test = shop_flc[['SHOP_CODE','SHOP_NAME','REGION_NAME_2','LATITUDE','LONGITUDE']] #lấy những cột cần chạy
filtered_get_test = get_test.dropna(subset=['LATITUDE', 'LONGITUDE'], how='all') #loại bỏ những shop có giá trị lat, long null
filtered_get_test

,SHOP_CODE,SHOP_NAME,REGION_NAME_2,LATITUDE,LONGITUDE
0,80006,LC HCM 72 Tân Mỹ,Hồ Chí Minh,10.735733,106.718295
1,80007,LC HCM 422-424 CMT8,Hồ Chí Minh,10.783764,106.670880
2,80008,LC HCM 94-96 Nguyễn Văn Nghi,Hồ Chí Minh,10.823002,106.688169
3,80009,LC HCM 1008-1010 Âu Cơ,Hồ Chí Minh,10.799065,106.637296
4,80010,LC HCM 501-503 Nguyễn Duy Trinh,Hồ Chí Minh,10.788630,106.769100
...,...,...,...,...,...
1249,81216,LC DTP 298 Quốc Lộ 30,Miền Tây,10.452917,105.653167
1250,81217,LC DTP 196 Điện Biên Phủ,Miền Tây,10.471435,105.645668
1251,81218,LC DNI 07 Nguyễn Tất Thành,Miền Đông,11.087453,107.033722
1264,82002,LC BDG 1178 ĐT747,Miền Đông,10.975333,106.764667


In [ ]:
session = requests.Session()
retries = Retry(total=5, backoff_factor=0.1, status_forcelist=[500, 502, 503, 504])
adapter = HTTPAdapter(max_retries=retries)
session.mount('http://', adapter)
session.mount('https://', adapter)

geolocator = Nominatim(user_agent="my_app", timeout=50)
geolocator.session = session

seen_osm_ids = []
seen_address = []
group_size = 10
df = pd.DataFrame()
SHOP_CODE = []
SHOP_NAME = []
CENTER_CORDINATES = []
POINTS_AROUND = []
LOCATION_ADDRESS = []
LOCATION_OSM_TYPE = []
LOCATION_OSM_ID = []
LOCATION_DISTANCE = []
for i in range(40, 50):
    shop_code = filtered_get_test['SHOP_CODE'][i]
    shop_name = filtered_get_test['SHOP_NAME'][i]
    center_coordinates = (filtered_get_test['LATITUDE'][i], filtered_get_test['LONGITUDE'][i])
    radius = 500  # Bán kính (đơn vị: mét)
    api = overpy.Overpass()  # Kết nối tới API Overpass
    result = api.query(f"""node(around:{radius},{center_coordinates[0]},{center_coordinates[1]});out;""")  # Truy vấn Overpass để lấy tọa độ xung quanh
    points_around = [(node.lat, node.lon) for node in result.nodes]  # Lấy danh sách tọa độ xung quanh
    list_point = []  # Danh sách tọa độ xung quanh
    for point in points_around:
        coordinates = (point[0].to_eng_string(), point[1].to_eng_string())
        list_point.append(coordinates)
    for i in tqdm(range(0, len(list_point), group_size)):
        group = list_point[i:i + group_size]
        for j in range(len(group)):
            # Lấy thông tin địa điểm từ tọa độ
            location = geolocator.reverse(group[j], exactly_one=True)
            distance = geodesic(center_coordinates, group[j]).meters
            osm_id = location.raw['osm_id']
            address = location.address
            # Kiểm tra nếu osm_id đã xuất hiện trong danh sách
            if osm_id not in seen_osm_ids and address not in seen_address:
                seen_osm_ids.append(osm_id)
                seen_address.append(address)
                SHOP_CODE.append(shop_code)
                SHOP_NAME.append(shop_name)
                CENTER_CORDINATES.append(center_coordinates)
                POINTS_AROUND.append(group[j])
                LOCATION_ADDRESS.append(location.address)
                LOCATION_OSM_TYPE.append(location.raw['osm_type'])
                LOCATION_OSM_ID.append(location.raw['osm_id'])
                LOCATION_DISTANCE.append(distance)

 36%|███▌      | 54/152 [04:32<08:09,  4.99s/it]

In [22]:
df['SHOP_CODE'] = pd.DataFrame(SHOP_CODE)
df['SHOP_NAME'] = pd.DataFrame(SHOP_NAME)
df['CENTER_CORDINATES'] = pd.Series(CENTER_CORDINATES).apply(lambda x: f"{x[0]}, {x[1]}")
df['POINTS_AROUND'] = pd.Series(POINTS_AROUND).apply(lambda x: f"{x[0]}, {x[1]}")
df['LOCATION_ADDRESS'] = pd.DataFrame(LOCATION_ADDRESS)
df['LOCATION_OSM_TYPE'] = pd.DataFrame(LOCATION_OSM_TYPE)
df['LOCATION_OSM_ID'] = pd.DataFrame(LOCATION_OSM_ID)
df['LOCATION_DISTANCE'] = pd.DataFrame(LOCATION_DISTANCE)
df.head(3)

,SHOP_CODE,SHOP_NAME,CENTER_CORDINATES,POINTS_AROUND,LOCATION_ADDRESS,LOCATION_OSM_TYPE,LOCATION_OSM_ID,LOCATION_DISTANCE
0,80037,LC TGG 134 Lý Thường Kiệt,"10.357079, 106.357369","10.3578194, 106.3563140","ATM BIDV, Lý Thường Kiệt, Phường 5, Thành phố ...",node,2741374883,141.621369
1,80037,LC TGG 134 Lý Thường Kiệt,"10.357079, 106.357369","10.3527832, 106.3565658","Trần Hưng Đạo, Phường 4, Thành phố Mỹ Tho, Tỉn...",way,111514144,483.232912
2,80037,LC TGG 134 Lý Thường Kiệt,"10.357079, 106.357369","10.3580578, 106.3617533","Trường Tiểu Học Thủ Khoa Huân, 54, Đường Nam K...",way,266578185,492.211482


In [23]:
api = osmapi.OsmApi()
TAGS = []
OSM_ID = []
OSM_TYPE = []
OSM_NAME = []
SHOP_NAME = []
SHOP_CODE = []
CENTER_CORDINATES = []
POINTS_AROUND = []
LOCATION_DISTANCE = []
for i in tqdm(range(len(df))):
    osm_id = df['LOCATION_OSM_ID'][i]
    if df['LOCATION_OSM_TYPE'][i] == 'node':
        object_info_NodeGet = api.NodeGet(osm_id)
        tags_NodeGet = object_info_NodeGet.get('tag', {})
        OSM_ID.append(df['LOCATION_OSM_ID'][i])
        OSM_TYPE.append(df['LOCATION_OSM_TYPE'][i])
        OSM_NAME.append(df['LOCATION_ADDRESS'][i])
        CENTER_CORDINATES.append(df['CENTER_CORDINATES'][i])
        POINTS_AROUND.append(df['POINTS_AROUND'][i])
        SHOP_NAME.append(df['SHOP_NAME'][i])
        SHOP_CODE.append(df['SHOP_CODE'][i])
        LOCATION_DISTANCE.append(df['LOCATION_DISTANCE'][i])
        TAGS.append(tags_NodeGet)
    elif df['LOCATION_OSM_TYPE'][i] == 'way':
        object_info_WayGet = api.WayGet(osm_id)
        tags_WayGet = object_info_WayGet.get('tag', {})
        OSM_ID.append(df['LOCATION_OSM_ID'][i])
        OSM_TYPE.append(df['LOCATION_OSM_TYPE'][i])
        OSM_NAME.append(df['LOCATION_ADDRESS'][i])
        CENTER_CORDINATES.append(df['CENTER_CORDINATES'][i])
        POINTS_AROUND.append(df['POINTS_AROUND'][i])
        SHOP_NAME.append(df['SHOP_NAME'][i])
        SHOP_CODE.append(df['SHOP_CODE'][i])
        LOCATION_DISTANCE.append(df['LOCATION_DISTANCE'][i])
        TAGS.append(tags_WayGet)
    elif df['LOCATION_OSM_TYPE'][i] == 'relation':
        object_info_RelationGet = api.RelationGet(osm_id)
        tags_RelationGet = object_info_RelationGet.get('tag', {})
        OSM_ID.append(df['LOCATION_OSM_ID'][i])
        OSM_TYPE.append(df['LOCATION_OSM_TYPE'][i])
        OSM_NAME.append(df['LOCATION_ADDRESS'][i])
        CENTER_CORDINATES.append(df['CENTER_CORDINATES'][i])
        POINTS_AROUND.append(df['POINTS_AROUND'][i])
        SHOP_NAME.append(df['SHOP_NAME'][i])
        SHOP_CODE.append(df['SHOP_CODE'][i])
        LOCATION_DISTANCE.append(df['LOCATION_DISTANCE'][i])
        TAGS.append(tags_RelationGet)

100%|██████████| 317/317 [01:37<00:00,  3.26it/s]


In [24]:
df2 = pd.DataFrame(TAGS)
df2['SHOP_NAME'] = pd.DataFrame(SHOP_NAME)
df2['SHOP_CODE'] = pd.DataFrame(SHOP_CODE)
df2['CENTER_CORDINATES'] = pd.DataFrame(CENTER_CORDINATES)
df2['POINTS_AROUND'] = pd.DataFrame(POINTS_AROUND)
df2['LOCATION_DISTANCE'] = pd.DataFrame(LOCATION_DISTANCE)
df2['OSM_ID'] = pd.DataFrame(OSM_ID)
df2['OSM_TYPE'] = pd.DataFrame(OSM_TYPE)
df2['OSM_NAME'] = pd.DataFrame(OSM_NAME)
df2.head(10)

,amenity,name,highway,oneway,addr:city,addr:housenumber,addr:postcode,addr:province,addr:street,addr:subdistrict,...,residential,addr:country,SHOP_NAME,SHOP_CODE,CENTER_CORDINATES,POINTS_AROUND,LOCATION_DISTANCE,OSM_ID,OSM_TYPE,OSM_NAME
0,atm,ATM BIDV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,LC TGG 134 Lý Thường Kiệt,80037,"10.357079, 106.357369","10.3578194, 106.3563140",141.621369,2741374883,node,"ATM BIDV, Lý Thường Kiệt, Phường 5, Thành phố ..."
1,NaN,Trần Hưng Đạo,tertiary,no,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,LC TGG 134 Lý Thường Kiệt,80037,"10.357079, 106.357369","10.3527832, 106.3565658",483.232912,111514144,way,"Trần Hưng Đạo, Phường 4, Thành phố Mỹ Tho, Tỉn..."
2,school,Trường Tiểu Học Thủ Khoa Huân,NaN,NaN,Mỹ Tho,54,861061,Tiền Giang,Nam Kỳ Khởi Nghĩa,Phường 1,...,NaN,NaN,LC TGG 134 Lý Thường Kiệt,80037,"10.357079, 106.357369","10.3580578, 106.3617533",492.211482,266578185,way,"Trường Tiểu Học Thủ Khoa Huân, 54, Đường Nam K..."
3,NaN,Yersin,tertiary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,LC TGG 134 Lý Thường Kiệt,80037,"10.357079, 106.357369","10.3581185, 106.3611096",425.490415,111315245,way,"Yersin, Phường 4, Thành phố Mỹ Tho, Tỉnh Tiền ..."
4,bank,VietinBank Tiền Giang,NaN,NaN,Mỹ Tho,NaN,NaN,Tiền Giang,Nam Kỳ Khởi Nghĩa,NaN,...,NaN,NaN,LC TGG 134 Lý Thường Kiệt,80037,"10.357079, 106.357369","10.3561975, 106.3615634",469.594906,267606559,way,"VietinBank Tiền Giang, Đường Nam Kỳ Khởi Nghĩa..."
5,cafe,Tiểu Lan Phố,NaN,NaN,Mỹ Tho,NaN,NaN,NaN,Yersin,NaN,...,NaN,NaN,LC TGG 134 Lý Thường Kiệt,80037,"10.357079, 106.357369","10.3550246, 106.3607292",432.507121,269790272,way,"Tiểu Lan Phố, Yersin, Phường 4, Thành phố Mỹ T..."
6,NaN,Nhà tưởng niệm Anh Hùng Liệt Sĩ,NaN,NaN,Mỹ Tho,NaN,NaN,Tiền Giang,Lý Thường Kiệt,4,...,NaN,NaN,LC TGG 134 Lý Thường Kiệt,80037,"10.357079, 106.357369","10.3547973, 106.3599598",379.740882,366901567,way,"Nhà tưởng niệm Anh Hùng Liệt Sĩ, Lý Thường Kiệ..."
7,NaN,Tết Mậu Thân,tertiary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,LC TGG 134 Lý Thường Kiệt,80037,"10.357079, 106.357369","10.3555998, 106.3600050",331.829811,266761755,way,"Tết Mậu Thân, Phường 4, Thành phố Mỹ Tho, Tỉnh..."
8,NaN,Thế Giới Di Động,NaN,NaN,Mỹ Tho,NaN,NaN,NaN,Lý Thường Kiệt,NaN,...,NaN,NaN,LC TGG 134 Lý Thường Kiệt,80037,"10.357079, 106.357369","10.3571540, 106.3571707",23.247838,3078023070,node,"Thế Giới Di Động, Lý Thường Kiệt, Phường 5, Th..."
9,NaN,Đống Đa,tertiary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,LC TGG 134 Lý Thường Kiệt,80037,"10.357079, 106.357369","10.3559608, 106.3587601",196.235486,223820455,way,"Đống Đa, Phường 4, Thành phố Mỹ Tho, Tỉnh Tiền..."


In [25]:
df2.to_excel(r'D:\SHOP_LC\shop_31_34_details.xlsx')